## This notebook compares performance of our models with original Baly's models

In [79]:
import importlib
import classification as nmrc
importlib.reload(nmrc)
import pandas as pd
from multiprocessing import Pool, Manager

In [2]:
news_media_data_folder_location='''../News-Media_Reliability/data/'''
news_media_corpus_file = news_media_data_folder_location+'''corpus.csv'''
news_media_feature_location = news_media_data_folder_location+'''features/'''

In [3]:
model1_feature_location = '''features/mimic_model_original.h5/'''
model2_feature_location = '''features/mimic_model_complex.h5/'''

In [74]:
def run_classification(result_list, name, feature_location, corpus = news_media_corpus_file
                       , task = 'bias', features = ['has_wiki'] ):
    result = nmrc.Classification(corpus, features, task,feature_location)
    result_dict = {'name':name
                    ,'feature_location':feature_location
                    ,'task':task
                    ,'features':",".join(features)
                    ,'F1':result[0]
                    ,'Accuracy':result[1]
                    ,'MAE':result[2]
                    ,'MAE_U':result[3]}
    
    result_list.append(result_dict)
    
    

In [ ]:
result_list=Manager().list()
tasks = ['bias','fact']
features = [['has_wiki']#, ['wikicontent'], ['wikisummary'], ['wikitoc'], ['wikicategories']
           ,['has_wiki','wikicontent','wikisummary','wikitoc','wikicategories']]
feature_locations = [news_media_feature_location,model1_feature_location,model2_feature_location]
model_names = ['original','original_replica','complex']
corpus = news_media_corpus_file
processes = []
parameters_list = []
no_of_cpu=4 # on gcp this can be increased to match capacity

def run_classifier():

    for i,model_name in enumerate(model_names):
        feature_location =feature_locations[i]
        for task in tasks:
            for feature_list in features:
                print('corpus',corpus)
                print('feature_list',feature_list)
                print('task',task)
                print('feature_location',feature_location)

                parameters = (result_list, model_name, feature_location, corpus, task, feature_list)
                parameters_list.append(parameters)
#                     p = Process(target=run_classification, args=parameters)
#                     p.start()
#                     processes.append(p)
    #             result = run_classification(model_name, feature_location, corpus, task, feature_list)
    #             result_list.append(result)
    #print(parameters_list)
    with Pool(processes=no_of_cpu) as pool:
        result_async = pool.starmap_async(run_classification, parameters_list)

        pool.close()
        pool.join()
        
    #print(result_async.get())

        
run_classifier()

corpus ../News-Media_Reliability/data/corpus.csv
feature_list ['has_wiki']
task bias
feature_location ../News-Media_Reliability/data/features/
corpus ../News-Media_Reliability/data/corpus.csv
feature_list ['has_wiki', 'wikicontent', 'wikisummary', 'wikitoc', 'wikicategories']
task bias
feature_location ../News-Media_Reliability/data/features/
corpus ../News-Media_Reliability/data/corpus.csv
feature_list ['has_wiki']
task fact
feature_location ../News-Media_Reliability/data/features/
corpus ../News-Media_Reliability/data/corpus.csv
feature_list ['has_wiki', 'wikicontent', 'wikisummary', 'wikitoc', 'wikicategories']
task fact
feature_location ../News-Media_Reliability/data/features/
corpus ../News-Media_Reliability/data/corpus.csv
feature_list ['has_wiki']
task bias
feature_location features/mimic_model_original.h5/
corpus ../News-Media_Reliability/data/corpus.csv
feature_list ['has_wiki', 'wikicontent', 'wikisummary', 'wikitoc', 'wikicategories']
task bias
feature_location features/mimi

In [93]:
#print(result_list)
result_df = pd.DataFrame(result_list[:])
display_result_df = result_df.drop(labels=['feature_location'], axis=1)
#display_result_df=display_result_df.sort_values(by=1)

FileNotFoundError: [Errno 2] No such file or directory

In [89]:
display_result_df.style

,Accuracy,F1,MAE,MAE_U,features,name,task
0,26.0788,10.1856,1.40056,1.7383,has_wiki,original_replica,bias
1,26.8293,14.3271,1.63133,2.13737,has_wiki,original,bias


### BaseLine

In [ ]:
corpus = news_media_corpus_file
task = 'bias'
features = ['wikicontent']#['has_wiki']#, 'wikicontent', 'wikisummary', 'wikitoc', 'wikicategories']
feature_location = news_media_feature_location

print('task:'     + task)
print('features:' + ', '.join(features))
results = nmrc.Classification(corpus, features, task,feature_location)

In [ ]:
print('Results:')
print('F1\t{}'.format(results[0]))
print('Acc.\t{}'.format(results[1]))
print('MAE\t{}'.format(results[2]))
print('MAE_u\t{}'.format(results[3]))

In [ ]:
model1_feature_location = '''features/mimic_model_original.h5/'''
corpus = news_media_corpus_file
task = 'bias'
features = ['wikicontent']#['has_wiki', 'wikicontent', 'wikisummary', 'wikitoc', 'wikicategories']
feature_location = news_media_feature_location

print('task:'     + task)
print('features:' + ', '.join(features))
results = nmrc.Classification(corpus, features, task,model1_feature_location)

In [ ]:
print('Results:')
print('F1\t{}'.format(results[0]))
print('Acc.\t{}'.format(results[1]))
print('MAE\t{}'.format(results[2]))
print('MAE_u\t{}'.format(results[3]))

In [ ]:
model2_feature_location = '''features/mimic_model_complex.h5/'''
corpus = news_media_corpus_file
task = 'bias'
features = ['has_wiki']#, 'wikicontent', 'wikisummary', 'wikitoc', 'wikicategories']
feature_location = news_media_feature_location

print('task:'     + task)
print('features:' + ', '.join(features))
results = nmrc.Classification(corpus, features, task,model2_feature_location)

In [ ]:
print('Results:')
print('F1\t{}'.format(results[0]))
print('Acc.\t{}'.format(results[1]))
print('MAE\t{}'.format(results[2]))
print('MAE_u\t{}'.format(results[3]))